In [1]:
# 搭建数据集

In [2]:
import numpy as np

In [3]:
dict_list = ["经常","经","有","有意见","意见","分歧","见","意","见分歧","分"]
prob = [0.1 ,0.05 ,0.1 ,0.1 ,0.2 ,0.2 ,0.05 ,0.05 ,0.05 ,0.1]
logits = [2.3 , 3 , 2.3 , 2.3 , 1.6 , 1.6 , 1 , 1 , 1, 2.3]

In [4]:
data = [dict_list,prob,logits]

In [5]:
data = np.array(data)

In [6]:
data

array([['经常', '经', '有', '有意见', '意见', '分歧', '见', '意', '见分歧', '分'],
       ['0.1', '0.05', '0.1', '0.1', '0.2', '0.2', '0.05', '0.05',
        '0.05', '0.1'],
       ['2.3', '3', '2.3', '2.3', '1.6', '1.6', '1', '1', '1', '2.3']],
      dtype='<U32')

In [7]:
demo = "经常有意见分歧"

In [8]:
# 搭建有向图 边代表词

In [9]:
V = [[ 0 for i in range(len(demo)+1)] for i in range(len(demo)+1)]

In [10]:
# 有向图预处理

In [11]:
def preprocess(V):
    for i in range(len(demo)):
        V[i][i+1] = 20
    return V

In [12]:
V = preprocess(V)

In [13]:
def update_V(data,demo,V):
    for i in range(len(demo)):
        maxNum =  i+1
        left = 0
        right = left+maxNum
        while(right<=len(demo)):
            temp = demo[left:right]
#             print(temp)
            for j in range(len(data[0])):
                if temp == data[0][j]:
                    V[left][right] = float(data[2][j])
            left +=1
            right = left+maxNum

In [14]:
update_V(data,demo,V)

In [15]:
V

[[0, 3.0, 2.3, 0, 0, 0, 0, 0],
 [0, 0, 20, 0, 0, 0, 0, 0],
 [0, 0, 0, 2.3, 0, 2.3, 0, 0],
 [0, 0, 0, 0, 1.0, 1.6, 0, 0],
 [0, 0, 0, 0, 0, 1.0, 0, 1.0],
 [0, 0, 0, 0, 0, 0, 2.3, 1.6],
 [0, 0, 0, 0, 0, 0, 0, 20],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [16]:
# 设置一个数组，存放到达当前节点最短路径的上一个节点号

In [17]:
nodes = [i+1 for i in range(len(demo))]

In [18]:
nodes

[1, 2, 3, 4, 5, 6, 7]

In [19]:
def dp_getMinRoute(V,nodes):
    nums = [1000 for i in range(len(V[0]))]
    nums[0] = 0
    queue = [0,]
    index = queue.pop(0)
    for i in range(len(V[index])):
        if V[index][i] != 0:
            queue.append(i)
            if nums[i]>V[index][i]:
                nums[i] = V[index][i]
                nodes[i-1] = index
    while(len(queue)!=0):
        index = queue.pop(0)
        for i in range(len(V[index])):
            if V[index][i] != 0:
                if i not in queue:
                    queue.append(i)
                if nums[i]>nums[index]+V[index][i]:
                    nums[i] = nums[index]+V[index][i]
                    nodes[i-1] = index
    return nodes

In [20]:
nodes = dp_getMinRoute(V,nodes)

In [21]:
# 解析最短路径

In [22]:
def decode_nodes(nodes):
    temp = nodes[len(nodes)-1]
    result = []
    result.append(len(nodes))
    result.append(temp)
    while(temp!=0):
        result.append(nodes[temp-1])
        temp = nodes[temp-1]
    result = result[::-1]
    return result

In [23]:
nodes_result = decode_nodes(nodes)

In [24]:
nodes_result

[0, 2, 5, 7]

In [25]:
# 分词切割

In [26]:
def getResult(demo,nodes):
    result = []
    for i in range(len(nodes)-1):
        result.append(demo[nodes[i]:nodes[i+1]])
    return result

In [27]:
getResult(demo,nodes_result)

['经常', '有意见', '分歧']